# Xarray development notebook

**Author:** Xavier R Nogueira

**Things to explore:**
* Can we dynamically feed slices of an xarray into numba JIT functions using `xarray.apply_ufunc`?

**Performance Notes:**
* Numba JIT is slower w/o compiling a loop.
* Parallelizing via numba is slower at 100k iters (for a simple equation), but slightly faster at 1M.
* JIT compiling AROUND a JIT compiled function to apply the loop is by far the fastest.
* Could we loop and blast thru writing results, throw them in a stack of sorts, and async write into xarray? Avoiding the writing bottle neck could be fast.
* Another idea is to loop over a dictionary of functions_names and params. Pop from the dict and add to another dict. Basically an ordered dict would control the flow. **I like this idea, but how can we make it work for xarray**.

**Construction Notes:**
* One can use `xarray.apply_ufunc` programatically via `*args` syntax. This means we can rapidly evaluate a time step.
* It might be faster to project all constants into xarray first, passing one in via the `*args` syntax slows things down, I'm assuming it's because each time it's broadcasting.

In [1]:
import numba
import xarray as xr
import clearwater_modules_python

In [2]:
dir(clearwater_modules_python)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'shared_equations',
 'tsm']

In [3]:
dir(clearwater_modules_python.tsm)

['EnergyBudget',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'constants',
 'equations',
 'model']

In [4]:
dir(clearwater_modules_python.tsm.equations)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'dTdt_sediment_c',
 'dTdt_water_c',
 'density_air',
 'emissivity_air',
 'mixing_ratio_air',
 'numba',
 'q_latent',
 'q_net',
 'q_sediment',
 'q_sensible',
 'wind_function']

In [5]:
clearwater_modules_python.tsm.equations.q_sensible.__annotations__

{'wind_kh_kw': float,
 'ri_function': float,
 'cp_air': float,
 'density_water': float,
 'wind_function': float,
 'air_temp_k': float,
 'water_temp_k': float,
 'return': float}

# Numba comparison

In [6]:
TEST_ITERS = 1000000

In [7]:
def q_sensible(
    wind_kh_kw: float,
    ri_function: float,
    cp_air: float,
    density_water: float,
    wind_function: float,
    air_temp_k: float,
    water_temp_k: float,
) -> float:
    # TODO: check if the return units are correct
    """Sensible heat flux (W/m2).

    Args:
        wind_kh_kw: Diffusivity ratio (unitless)
        ri_function: Richardson number (unitless)
        cp_air: Specific heat of air (J/kg/K)
        density_water: Water density (kg/m^3)
        wind_function: Wind function (unitless)
        air_temp_k: Air temperature (K)
        water_temp_k: Water temperature (K)
    """
    return (
        wind_kh_kw *
        ri_function *
        cp_air * density_water * wind_function *
        (air_temp_k - water_temp_k)
    )

In [8]:
%%time
for i in range(TEST_ITERS):
    clearwater_modules_python.tsm.equations.q_sensible(
        float(i),
        float(i*2),
        float(i*3),
        float(i*4),
        float(i*5),
        float(i*6),
        float(i*7),
    )

CPU times: total: 1.33 s
Wall time: 1.55 s


In [9]:
%%time
for i in range(TEST_ITERS):
    q_sensible(
        float(i),
        float(i*2),
        float(i*3),
        float(i*4),
        float(i*5),
        float(i*6),
        float(i*7),
    )

CPU times: total: 781 ms
Wall time: 1.02 s


In [33]:
%%time
@numba.njit
def iter_numba(func: callable):
    for i in range(TEST_ITERS):
        func(
            float(i),
            float(i*2),
            float(i*3),
            float(i*4),
            float(i*5),
            float(i*6),
            float(i*7),
        )
iter_numba(clearwater_modules_python.tsm.equations.q_sensible)

CPU times: total: 109 ms
Wall time: 138 ms


In [12]:
ds = xr.tutorial.open_dataset('air_temperature')
ds

<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [36]:
test_list = [
        ds.air,
        ds.air*2,
        ds.air*3,
        ds.air*4,
        ds.air*5,
        ds.air*6,
        ds.air*6,
]

In [37]:
%%time
xr.apply_ufunc(
        clearwater_modules_python.tsm.equations.q_sensible,
        *test_list,
    )

CPU times: total: 62.5 ms
Wall time: 91.8 ms


<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00

In [30]:
%%time
xr.apply_ufunc(
        clearwater_modules_python.tsm.equations.q_sensible,
        ds.air,
        ds.air*2,
        ds.air*3,
        ds.air*4,
        ds.air*5,
        ds.air*6,
        ds.air*7,
        dask='parrallelized', 
    )

CPU times: total: 125 ms
Wall time: 130 ms


<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[-2.3628990e+16, -2.4403478e+16, -2.5013533e+16, ...,
         -1.9102000e+16, -2.0470418e+16, -2.2141336e+16],
        [-2.5198994e+16, -2.5636243e+16, -2.5762333e+16, ...,
         -1.9102000e+16, -2.0366323e+16, -2.2533943e+16],
        [-2.9296876e+16, -2.9156519e+16, -2.8525019e+16, ...,
         -1.9299792e+16, -2.0938996e+16, -2.3924411e+16],
        ...,
        [-8.1697356e+16, -8.1038500e+16, -8.1367425e+16, ...,
         -7.9734144e+16, -7.9249457e+16, -7.8607110e+16],
        [-8.0547336e+16, -8.1038500e+16, -8.2011919e+16, ...,
         -8.0547336e+16, -8.0547336e+16, -7.9410724e+16],
        [-8.1186410e+16, -8.2011919e+16, -8.2527178e+16, ...,
         -8.2194481e+16, -8.2011919e+16, -8.1697356e+16]],

       [[-2.4162947e+16, -2.4524491e+16, -2.4767998e+16, ...,
         -1.8711523e+16, -1.9499263e+16, -2.0627369e+16],
        [-2.5075226e+16, -2.5385621e+16, -2.5448096e+16, ...,
         -1.8232789e+16, -1.8954790e+16, -2.0574953e+16],
        [-3.1620125e+16, -3.1388544e+16, -3.0804787e+16, ...,
         -1.8138268e+16, -1.9394326e+16, -2.2085718e+16],
...
        [-7.7004537e+16, -7.7319702e+16, -7.9717934e+16, ...,
         -7.9233342e+16, -7.8591167e+16, -7.7953278e+16],
        [-8.1186410e+16, -8.2677339e+16, -8.3347190e+16, ...,
         -7.9556160e+16, -7.9233342e+16, -7.8112338e+16],
        [-8.3683933e+16, -8.4700681e+16, -8.4871105e+16, ...,
         -8.0204949e+16, -7.9879975e+16, -7.9394652e+16]],

       [[-2.6009661e+16, -2.5504407e+16, -2.4884366e+16, ...,
         -2.3918467e+16, -2.3799951e+16, -2.3977896e+16],
        [-2.9219616e+16, -2.8801175e+16, -2.8182912e+16, ...,
         -2.2698297e+16, -2.3099109e+16, -2.3918467e+16],
        [-3.9702381e+16, -3.8983253e+16, -3.8274979e+16, ...,
         -2.2869365e+16, -2.4457863e+16, -2.6783156e+16],
        ...,
        [-7.7161991e+16, -7.7004537e+16, -7.9233342e+16, ...,
         -7.9556160e+16, -7.9233342e+16, -7.8591167e+16],
        [-8.0858085e+16, -8.2177843e+16, -8.2677339e+16, ...,
         -8.0204949e+16, -8.0204949e+16, -7.9394652e+16],
        [-8.3515450e+16, -8.4190954e+16, -8.4190954e+16, ...,
         -8.1515730e+16, -8.1022136e+16, -8.0204949e+16]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00

In [15]:
out

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[-2.3628990e+16, -2.4403478e+16, -2.5013533e+16, ...,
         -1.9102000e+16, -2.0470418e+16, -2.2141336e+16],
        [-2.5198994e+16, -2.5636243e+16, -2.5762333e+16, ...,
         -1.9102000e+16, -2.0366323e+16, -2.2533943e+16],
        [-2.9296876e+16, -2.9156519e+16, -2.8525019e+16, ...,
         -1.9299792e+16, -2.0938996e+16, -2.3924411e+16],
        ...,
        [-8.1697356e+16, -8.1038500e+16, -8.1367425e+16, ...,
         -7.9734144e+16, -7.9249457e+16, -7.8607110e+16],
        [-8.0547336e+16, -8.1038500e+16, -8.2011919e+16, ...,
         -8.0547336e+16, -8.0547336e+16, -7.9410724e+16],
        [-8.1186410e+16, -8.2011919e+16, -8.2527178e+16, ...,
         -8.2194481e+16, -8.2011919e+16, -8.1697356e+16]],

       [[-2.4162947e+16, -2.4524491e+16, -2.4767998e+16, ...,
         -1.8711523e+16, -1.9499263e+16, -2.0627369e+16],
        [-2.5075226e+16, -2.5385621e+16, -2.5448096e+16, ...,
         -1.8232789e+16, -1.8954790e+16, -2.0574953e+16],
        [-3.1620125e+16, -3.1388544e+16, -3.0804787e+16, ...,
         -1.8138268e+16, -1.9394326e+16, -2.2085718e+16],
...
        [-7.7004537e+16, -7.7319702e+16, -7.9717934e+16, ...,
         -7.9233342e+16, -7.8591167e+16, -7.7953278e+16],
        [-8.1186410e+16, -8.2677339e+16, -8.3347190e+16, ...,
         -7.9556160e+16, -7.9233342e+16, -7.8112338e+16],
        [-8.3683933e+16, -8.4700681e+16, -8.4871105e+16, ...,
         -8.0204949e+16, -7.9879975e+16, -7.9394652e+16]],

       [[-2.6009661e+16, -2.5504407e+16, -2.4884366e+16, ...,
         -2.3918467e+16, -2.3799951e+16, -2.3977896e+16],
        [-2.9219616e+16, -2.8801175e+16, -2.8182912e+16, ...,
         -2.2698297e+16, -2.3099109e+16, -2.3918467e+16],
        [-3.9702381e+16, -3.8983253e+16, -3.8274979e+16, ...,
         -2.2869365e+16, -2.4457863e+16, -2.6783156e+16],
        ...,
        [-7.7161991e+16, -7.7004537e+16, -7.9233342e+16, ...,
         -7.9556160e+16, -7.9233342e+16, -7.8591167e+16],
        [-8.0858085e+16, -8.2177843e+16, -8.2677339e+16, ...,
         -8.0204949e+16, -8.0204949e+16, -7.9394652e+16],
        [-8.3515450e+16, -8.4190954e+16, -8.4190954e+16, ...,
         -8.1515730e+16, -8.1022136e+16, -8.0204949e+16]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00